# Extraindo a série histórica de um subconjunto dos dados

Pode ser que seu interesse, ao trabalhar com os microdados das matrículas, seja selecionar determinada série histórica.

Os arquivos utilizados podem ser acessados no [Portal de Dados Abertos](http://dados.prefeitura.sp.gov.br/dataset/microdados-da-rede-municipal-de-ensino-matriculas). 

É possível que você necessite de só alguns dos anos, ou um intervalo menor. Por isso, nem sempre é interessante trabalhar com a base completa -- as operações podem ficar muito lentas ou mesmo inviáveis, por insuficiência de memória.

[Em um outro exercício](https://github.com/prefeiturasp/dados-educacao/blob/master/Tutorial%20-%20Estudantes%20Estrangeiros/Estudantes_estrangeiros.ipynb), fizemos a operação para a base de alunos estrangeiros, por nacionalidade, para apenas um ano letivo. Neste, faremos algumas operações para extrair toda a série histórica disponível.  

[Saiba mais](https://github.com/prefeiturasp/dados-educacao/blob/master/README.md) sobre os tutoriais dos dados abertos da SME.

*Importante: também estamos aprendendo! Tem sugestões para melhorar os códigos? Não deixe de contribuir com este repositório :)*

In [21]:
import pandas as pd

Trabalhamos aqui com os arquivos .csv disponíveis no Portal de Dados Abertos, já descompactados. Diferente do outro tutorial, porém, importaremos apenas algumas colunas (com que vamos trabalhar), para evitar insuficiência de memória: 

In [22]:
anos = ['2009','2010', '2011', '2012', '2013', '2014', '2015', '2016']
#anos letivos que serão selecionados

colunas = ['AN_LETIVO', 'CD_ALUNO_SME', 'SITUACAO_MAT', 'DESC_RACA_COR', 'CD_SEXO', 'MES_NASC_ALUNO', 
           'ANO_NASC_ALUNO', 'TIPO_ESCOLA', 'DRE', 'NOME_DISTRITO', 'TURMA_ESCOL', 
           'MODALIDADE_SEGMENTO', 'DESC_SERIE', 'DESC_PAIS_NASC']
#variáveis que serão selecionadas

A lógica aqui é fazer as operações com cada base de uma vez, filtrando as colunas desejadas (aqui pegamos as mesmas de outro exercício, selecionando as nacionalidades de estudantes estrangeiros). Passamos de bases de dados de 1GB, cada, para arquivos de aproximadamente 300KB.

### Opção 1. Salvando em arquivos diferentes

In [28]:
for ano in anos:
    df = pd.read_csv('Microdados_EOL_Matriculas_'+str(ano)+'.csv', sep=';', encoding='ISO-8859-1', low_memory=False, usecols=colunas)
    #Lê o arquivo e transforma em DataFrame
    
    estrangeiros = df[(df['DESC_PAIS_NASC'] != 'BRASIL')
                      & (df['TURMA_ESCOL'] == 'S') 
                      & (df['SITUACAO_MAT'] == 'Concluido')].dropna(how='any', subset=['DESC_PAIS_NASC']).drop_duplicates('CD_ALUNO_SME')
    #Filtra as nacionalidades diferentes de brasileira de alunos em turmas regulares e exclui registros em que as nacionalidades não estão declaradas
    
    estrangeiros.to_csv('estrangeiros'+str(ano)+'.csv', index=False)
    #Exporta para arquivo excel

Com o código acima, os arquivos '.csv' já foram gravados em arquivos separados no mesmo diretório.

### Opção 2. Salvando os arquivos em uma base única

In [27]:
df_todos = []

for ano in anos:
    
    df = pd.read_csv('Microdados_EOL_Matriculas_'+str(ano)+'.csv', sep=';', encoding='ISO-8859-1', low_memory=False, usecols=colunas)
    #Lê o arquivo e transforma em DataFrame
    
    estrangeiros = df[(df['DESC_PAIS_NASC'] != 'BRASIL')
                      & (df['TURMA_ESCOL'] == 'S') 
                      & (df['SITUACAO_MAT'] == 'Concluido')].dropna(how='any', subset=['DESC_PAIS_NASC']).drop_duplicates('CD_ALUNO_SME')
    #Filtra as nacionalidades diferentes de brasileira de alunos em turmas regulares e exclui registros em que as nacionalidades não estão declaradas
    
    df_todos.append(estrangeiros)

df_todos = pd.concat(df_todos, axis=0)

df_todos.to_excel('estrangeiros_completa.xlsx', index=False)

Pronto. O arquivo completo está salvo no mesmo diretório. 